In [1]:
#Preparación de datos
import pandas as pd
import numpy as np

## Definir las Etiquetas Binarias (Crisis vs No Crisis)
Dado que Rating es un valor de 1 a 5, se asignan las etiquetas de la siguiente forma:

Crisis (1) → Reseñas con puntuaciones bajas (por ejemplo, 1, 2 y 3).
No Crisis (0) → Reseñas con puntuaciones altas (por ejemplo, 4 y 5).

In [2]:
def assign_label(rating):
    if rating <= 3:
        return 1  # Crisis
    elif rating >= 4:
        return 0  # No crisis
    else:
        return None  # Opcional: descartar los neutros

In [3]:
# Unir los Vectores con las Etiquetas
# Cargar los archivos
df_vectors = pd.read_csv("review_vectors.csv")  # Archivo con los vectores
df_reviews = pd.read_csv("lemmatized_reviews.csv")  # Archivo con las reseñas lemmatizadas y calificaciones

# Asegurar que ambos archivos tienen un identificador común ('review_id')
df_model = df_reviews.merge(df_vectors, on="Review Id")

# Aplicar la función para asignar etiquetas
df_model["label"] = df_model["Rating"].apply(assign_label)

# Eliminar reseñas con etiqueta None (si es que hay algún rating vacío)
df_model = df_model.dropna(subset=["label"])

# Convertir la etiqueta a entero
df_model["label"] = df_model["label"].astype(int)

# Revisar el balance de clases
print(df_model["label"].value_counts())


label
0    35764
1     5292
Name: count, dtype: int64


In [4]:
df_model.head(5)

,Hotel_Name,Review Id,User Location,User Is Verified,Rating,Published Date,Language,Review Title,Review Text,Review Text Lemmatized,...,91,92,93,94,95,96,97,98,99,label
0,100 luxry,979285756,NaN,No,4,2024-11-09,es,a great place to share a spectacular night wit...,a very comfortable place;spacious;clean.excell...,a very comfortable place;spacious;clean.excell...,...,-0.231488,-0.437924,0.116230,0.647778,0.186457,0.152014,-0.621785,-0.397761,0.237131,0
1,100 luxry,906758315,NaN,No,5,2023-07-30,es,"incredible spanish dinner, super recommended",the room was decorated very beautiful from adm...,the room be decorate very beautiful from admis...,...,-0.221138,0.099882,0.232915,0.331478,0.508873,0.130649,-0.595498,-0.635350,0.078168,0
2,100 luxry,906479966,NaN,No,5,2023-07-29,es,excellent service,the bed is very comfortable;suitable for sleep...,the bed be very comfortable;suitable for sleep...,...,-0.294849,-0.466356,0.069100,0.096839,0.285948,-0.162023,-0.378548,-0.446968,-0.084183,0
3,100 luxry,895423330,NaN,No,3,2023-06-16,es,good hotel for short travel,the attention of all those who work at the hot...,the attention of all those who work at the hot...,...,-0.372905,-0.334735,0.739071,0.226722,0.471129,-0.154806,-0.671297,-0.517716,0.014172,1
4,100 luxry,890106560,Barranquilla,No,5,2023-05-12,es,super recommended.,spectacular attention.the first rooms.very com...,spectacular attention.the first rooms.very com...,...,-0.428489,0.145505,-0.176563,0.123192,0.077289,-0.131097,-0.483199,-0.463183,0.298192,0


In [5]:
# Verificar los nombres de las columnas
print(df_model.columns)
print(type(df_model.columns[0]))

Index(['Hotel_Name', 'Review Id', 'User Location', 'User Is Verified',
       'Rating', 'Published Date', 'Language', 'Review Title', 'Review Text',
       'Review Text Lemmatized',
       ...
       '91', '92', '93', '94', '95', '96', '97', '98', '99', 'label'],
      dtype='object', length=111)
<class 'str'>


## Entrenamiento del modelo
### Preparar los Datos para el Modelo
Se extraen las columnas necesarias:
vectors → Representación numérica de cada reseña
label → La etiqueta binaria.

In [6]:
# Extraer solo las columnas de vectores  
# Excluye la última columna (label)
X = np.array(df_model.iloc[:, df_model.columns.get_loc("0"):-1])

y = df_model["label"].values  # Etiquetas (1 = crisis, 0 = no crisis)

# Dividir en conjunto de entrenamiento y prueba (80% - 20%)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Tamaño de entrenamiento: {X_train.shape}, Tamaño de prueba: {X_test.shape}")


Tamaño de entrenamiento: (32844, 100), Tamaño de prueba: (8212, 100)


### Elegir un algoritmo de clasificación
Para este problema de clasificación binaria (crisis = 1, no crisis = 0), se usan los siguientes algoritmos:

Regresión Logística (rápido y explicable)

Random Forest (mejor para capturar relaciones no lineales)

SVM (efectivo si los datos no son linealmente separables)

Redes Neuronales MLPClassifier

In [7]:
# Regresión Logística
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Inicializar el modelo
model_LR = LogisticRegression(max_iter=1000, random_state=42)

# Entrenar el modelo
model_LR.fit(X_train, y_train)

# Predicciones
y_pred = model_LR.predict(X_test)

# Evaluar el modelo
accuracy = accuracy_score(y_test, y_pred)
print(f"Exactitud del modelo LR: {accuracy:.4f}")

# Reporte de clasificación
print("\nReporte de Clasificación LR:")
print(classification_report(y_test, y_pred))

# Matriz de confusión
print("\nMatriz de Confusión LR:")
print(confusion_matrix(y_test, y_pred))


Exactitud del modelo LR: 0.9149

Reporte de Clasificación LR:
              precision    recall  f1-score   support

           0       0.94      0.97      0.95      7143
           1       0.71      0.58      0.64      1069

    accuracy                           0.91      8212
   macro avg       0.83      0.77      0.80      8212
weighted avg       0.91      0.91      0.91      8212


Matriz de Confusión LR:
[[6894  249]
 [ 450  619]]


In [8]:
# Random Forest
from sklearn.ensemble import RandomForestClassifier

# Inicializar el modelo
model_RF = RandomForestClassifier(n_estimators=100, random_state=42)

# Entrenar el modelo
model_RF.fit(X_train, y_train)

# Predicciones
y_pred = model_RF.predict(X_test)

# Evaluar el modelo
accuracy = accuracy_score(y_test, y_pred)
print(f"Exactitud del modelo RF: {accuracy:.4f}")

# Reporte de clasificación
print("\nReporte de Clasificación RF:")
print(classification_report(y_test, y_pred))

# Matriz de confusión
print("\nMatriz de Confusión RF:")
print(confusion_matrix(y_test, y_pred))

Exactitud del modelo RF: 0.9062

Reporte de Clasificación RF:
              precision    recall  f1-score   support

           0       0.92      0.97      0.95      7143
           1       0.72      0.45      0.56      1069

    accuracy                           0.91      8212
   macro avg       0.82      0.71      0.75      8212
weighted avg       0.90      0.91      0.90      8212


Matriz de Confusión RF:
[[6956  187]
 [ 583  486]]


In [9]:
# SVM
from sklearn.svm import SVC
# Inicializar el modelo
model_SVC = SVC(kernel="linear", probability=True, random_state=42)

# Entrenar el modelo
model_SVC.fit(X_train, y_train)

# Predicciones
y_pred = model_SVC.predict(X_test)

# Evaluar el modelo
accuracy = accuracy_score(y_test, y_pred)
print(f"Exactitud del modelo SVC: {accuracy:.4f}")

# Reporte de clasificación
print("\nReporte de Clasificación SVC:")
print(classification_report(y_test, y_pred))

# Matriz de confusión
print("\nMatriz de Confusión SVC:")
print(confusion_matrix(y_test, y_pred))

Exactitud del modelo SVC: 0.9163

Reporte de Clasificación SVC:
              precision    recall  f1-score   support

           0       0.94      0.97      0.95      7143
           1       0.72      0.59      0.65      1069

    accuracy                           0.92      8212
   macro avg       0.83      0.78      0.80      8212
weighted avg       0.91      0.92      0.91      8212


Matriz de Confusión SVC:
[[6894  249]
 [ 438  631]]


In [10]:
# Red Neuronal (MLPClassifier)
from sklearn.neural_network import MLPClassifier

# Inicializar el modelo
model_MLP = MLPClassifier(hidden_layer_sizes=(100,100), max_iter=500, random_state=42)

# Entrenar el modelo
model_MLP.fit(X_train, y_train)

# Predicciones
y_pred = model_MLP.predict(X_test)

# Evaluar el modelo
accuracy = accuracy_score(y_test, y_pred)
print(f"Exactitud del modelo MLP: {accuracy:.4f}")

# Reporte de clasificación
print("\nReporte de Clasificación MLP:")
print(classification_report(y_test, y_pred))

# Matriz de confusión
print("\nMatriz de Confusión MLP:")
print(confusion_matrix(y_test, y_pred))

Exactitud del modelo MLP: 0.9009

Reporte de Clasificación MLP:
              precision    recall  f1-score   support

           0       0.94      0.94      0.94      7143
           1       0.62      0.63      0.62      1069

    accuracy                           0.90      8212
   macro avg       0.78      0.79      0.78      8212
weighted avg       0.90      0.90      0.90      8212


Matriz de Confusión MLP:
[[6725  418]
 [ 396  673]]


### Ajustar Hiperparámetros con GridSearchCV

In [11]:
from sklearn.model_selection import GridSearchCV
#from sklearn.linear_model import LogisticRegression

# Definir el modelo
lr = LogisticRegression()

# Definir los hiperparámetros a probar
param_grid = {
    "C": [0.01, 0.1, 1, 10, 100],  # Regularización
    "solver": ["liblinear", "lbfgs"]  # Métodos de optimización
}

# Configurar GridSearchCV
grid_search = GridSearchCV(lr, param_grid, cv=5, scoring="accuracy", n_jobs=-1)

# Entrenar el modelo con GridSearch
grid_search.fit(X_train, y_train)

# Mostrar mejores parámetros y exactitud
print("Mejores parámetros:", grid_search.best_params_)
print("Mejor exactitud:", grid_search.best_score_)

# Evaluar en conjunto de prueba
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)

from sklearn.metrics import classification_report, confusion_matrix
print("Reporte de Clasificación:\n", classification_report(y_test, y_pred))
print("Matriz de Confusión:\n", confusion_matrix(y_test, y_pred))


Mejores parámetros: {'C': 1, 'solver': 'lbfgs'}
Mejor exactitud: 0.9185848147343973
Reporte de Clasificación:
               precision    recall  f1-score   support

           0       0.94      0.97      0.95      7143
           1       0.71      0.58      0.64      1069

    accuracy                           0.91      8212
   macro avg       0.83      0.77      0.80      8212
weighted avg       0.91      0.91      0.91      8212

Matriz de Confusión:
 [[6894  249]
 [ 450  619]]


In [12]:
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix

# Definir el modelo
rf = RandomForestClassifier(random_state=42, n_jobs=-1)

# Definir el grid de hiperparámetros a probar
param_grid = {
    'n_estimators': [50, 100, 200],  # Número de árboles
    'max_depth': [10, 20, None],  # Profundidad máxima de los árboles
    'min_samples_split': [2, 5, 10],  # Mínimas muestras para dividir nodo
    'min_samples_leaf': [1, 2, 4]  # Mínimas muestras en una hoja
}

# Configurar GridSearchCV
grid_search_rf = GridSearchCV(
    estimator=rf, 
    param_grid=param_grid, 
    cv=5,  # Validación cruzada con 5 folds
    scoring='recall',  # Maximizar recall para detectar crisis
    n_jobs=-1,  # Usar todos los núcleos del procesador
    verbose=2
)

# Ajustar GridSearchCV al conjunto de entrenamiento
grid_search_rf.fit(X_train, y_train)

# Mostrar los mejores parámetros encontrados
print("Mejores parámetros:", grid_search_rf.best_params_)

# Evaluar el mejor modelo encontrado
best_rf = grid_search_rf.best_estimator_
y_pred_rf = best_rf.predict(X_test)

# Reporte de clasificación y matriz de confusión
print("Reporte de Clasificación Random Forest:")
print(classification_report(y_test, y_pred_rf))

print("Matriz de Confusión Random Forest:")
print(confusion_matrix(y_test, y_pred_rf))


Fitting 5 folds for each of 81 candidates, totalling 405 fits
Mejores parámetros: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 50}
Reporte de Clasificación Random Forest:
              precision    recall  f1-score   support

           0       0.92      0.97      0.95      7143
           1       0.70      0.46      0.55      1069

    accuracy                           0.90      8212
   macro avg       0.81      0.71      0.75      8212
weighted avg       0.89      0.90      0.90      8212

Matriz de Confusión Random Forest:
[[6934  209]
 [ 579  490]]


In [13]:
from sklearn.svm import SVC
# from sklearn.model_selection import GridSearchCV

# Definir el modelo base
svm_model = SVC()

# Definir los hiperparámetros a optimizar
param_grid = {
    'C': [0.1, 1, 10, 100],  # Regularización
    'kernel': ['linear', 'rbf', 'poly', 'sigmoid'],  # Diferentes funciones kernel
    'gamma': ['scale', 'auto'],  # Parámetro gamma (relevante para rbf, poly, y sigmoid)
}

# Aplicar GridSearchCV
grid_search_svm = GridSearchCV(svm_model, param_grid, cv=5, scoring='recall', verbose=2, n_jobs=-1)
grid_search_svm.fit(X_train, y_train)

# Mostrar los mejores parámetros encontrados
print("Mejores parámetros:", grid_search_svm.best_params_)

# Evaluar el mejor modelo
best_svm = grid_search_svm.best_estimator_
y_pred_svm = best_svm.predict(X_test)

# Reporte de clasificación y matriz de confusión
print("Reporte de Clasificación SVM:")
print(classification_report(y_test, y_pred_svm))

print("Matriz de Confusión SVM:")
print(confusion_matrix(y_test, y_pred_svm))


Fitting 5 folds for each of 32 candidates, totalling 160 fits
Mejores parámetros: {'C': 100, 'gamma': 'scale', 'kernel': 'rbf'}
Reporte de Clasificación SVM:
              precision    recall  f1-score   support

           0       0.95      0.96      0.95      7143
           1       0.71      0.65      0.68      1069

    accuracy                           0.92      8212
   macro avg       0.83      0.80      0.82      8212
weighted avg       0.92      0.92      0.92      8212

Matriz de Confusión SVM:
[[6856  287]
 [ 376  693]]


In [14]:
#from sklearn.neural_network import MLPClassifier
# from sklearn.model_selection import GridSearchCV

# Definir el modelo base
mlp = MLPClassifier(max_iter=500, random_state=42)

# Definir la cuadrícula de hiperparámetros
param_grid = {
    'hidden_layer_sizes': [(50,), (100,), (50, 50), (100, 50)],  # Número de neuronas en cada capa
    'activation': ['relu', 'tanh', 'logistic'],  # Función de activación
    'solver': ['adam', 'sgd'],  # Algoritmo de optimización
    'alpha': [0.0001, 0.001, 0.01],  # Regularización
    'learning_rate': ['constant', 'adaptive']  # Estrategia de tasa de aprendizaje
}

# Aplicar GridSearchCV con validación cruzada de 5 folds
grid_search_mlp = GridSearchCV(mlp, param_grid, cv=5, n_jobs=-1, verbose=3)

# Ajustar el modelo a los datos de entrenamiento
grid_search_mlp.fit(X_train, y_train)

# Mostrar los mejores hiperparámetros encontrados
print("Mejores parámetros:", grid_search_mlp.best_params_)

# Evaluar el modelo con los mejores parámetros
y_pred_mlp = grid_search_mlp.best_estimator_.predict(X_test)

# Generar métricas
from sklearn.metrics import classification_report, confusion_matrix

print("Reporte de Clasificación MLP:")
print(classification_report(y_test, y_pred_mlp))

print("Matriz de Confusión MLP:")
print(confusion_matrix(y_test, y_pred_mlp))


Fitting 5 folds for each of 144 candidates, totalling 720 fits
Mejores parámetros: {'activation': 'relu', 'alpha': 0.001, 'hidden_layer_sizes': (50, 50), 'learning_rate': 'adaptive', 'solver': 'sgd'}
Reporte de Clasificación MLP:
              precision    recall  f1-score   support

           0       0.95      0.97      0.96      7143
           1       0.74      0.63      0.68      1069

    accuracy                           0.92      8212
   macro avg       0.84      0.80      0.82      8212
weighted avg       0.92      0.92      0.92      8212

Matriz de Confusión MLP:
[[6908  235]
 [ 393  676]]
